In [1]:
# Standard imports:

### Please do not mind the keyboard Interrupts if any. They were used to ensure that the cell was running.

In [ ]:
import numpy as np
import scipy #only sparse matrices allowed
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

In [2]:
from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix
from sklearn.metrics import accuracy_score
#to deal with imbalanced classes
from sklearn.metrics import balanced_accuracy_score
from sklearn.metrics import f1_score
from sklearn.metrics import roc_auc_score
from sklearn.metrics import precision_score
from sklearn.metrics import recall_score

In [3]:
from sklearn.model_selection import GridSearchCV
from sklearn.experimental import enable_halving_search_cv 
from sklearn.model_selection import HalvingRandomSearchCV

In [4]:
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.naive_bayes import GaussianNB
from sklearn.naive_bayes import MultinomialNB
from sklearn.linear_model import SGDClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import GradientBoostingClassifier
from lightgbm import LGBMClassifier
from xgboost.sklearn import XGBClassifier

In [5]:
#We need to increase the categorization accuracy.

In [6]:
df_train = pd.read_csv(r"./ift3395-6390-weatherevents/train.csv")

In [7]:
submit_df = pd.read_csv(r"./ift3395-6390-weatherevents/test.csv")

In [8]:
df_train.head()

,S.No,lat,lon,TMQ,U850,V850,UBOT,VBOT,QREFHT,PS,...,T200,T500,PRECT,TS,TREFHT,Z1000,Z200,ZBOT,time,LABELS
0,0,-24.758801,242.1875,16.019615,-4.391696,4.777769,-6.388222,7.725320,0.010277,102326.2344,...,221.594223,261.823212,6.660000e-09,295.622253,293.301514,197.698624,12234.78809,64.372635,20030804,0
1,1,23.820078,277.8125,47.802036,8.623652,9.308566,4.596105,9.938286,0.018531,100803.1094,...,220.343719,269.106720,3.670000e-16,302.496826,302.654083,71.571518,12463.97266,66.834389,20071006,1
2,2,23.820078,276.8750,11.556691,-2.483993,-6.009627,-3.503036,-5.921963,0.007468,101885.3906,...,215.933487,259.610718,0.000000e+00,299.854919,293.339996,160.414246,12099.36133,64.209663,20020331,0
3,3,13.494133,253.1250,53.186630,0.150933,-1.319407,3.757741,-2.172120,0.018809,100824.6328,...,218.457672,267.374329,9.800000e-08,301.606934,299.973450,72.840614,12379.26855,66.243294,20000906,0
4,4,-24.524120,241.2500,23.353998,-7.467506,-5.113565,-9.545109,-4.691221,0.011672,102553.8984,...,214.316605,263.575409,6.410000e-09,295.819488,295.101257,218.449570,12261.22656,64.876427,20000906,0


In [9]:
def df_year_handler(df_train):
    #df_train is the train df loaded from our csv file
    #labels are not dealt with here, so keep them in a separate column before calling this
    #df_train2 = df_train.drop(columns=["S.No"])
    df_train2 = df_train
    #df_train2 = df_train
    #https://stackoverflow.com/questions/22005911/convert-columns-to-string-in-pandas
    #converting to string for the split
    df_train2["time"] = df_train2["time"].astype(str)
    #expand will return multiple columns with the split strings
    split_columns = df_train2["time"].str.split("",expand=True)
    year = split_columns[1] + split_columns[2] + split_columns[3] +split_columns[4]
    month = split_columns[5] + split_columns[6]
    date = split_columns[7] + split_columns[8]
    
    #converting series to df and renaming the column so we dont end up with a bunch of columns named 0
    #https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.rename.html
    year = pd.DataFrame(year)
    month = pd.DataFrame(month)
    date = pd.DataFrame(date)

    year = year.rename(columns={0: "year"})
    month = month.rename(columns={0: "month"})
    date = date.rename(columns={0: "date"})

    #we need to convert the strings back to integers
    #https://datatofish.com/string-to-integer-dataframe/
    year['year'] = year['year'].astype(int)
    month['month'] = month['month'].astype(int)
    date['date'] = date['date'].astype(int)
    
    #adding these back to the original df
    df = pd.concat([df_train2, year, month, date], axis = 1)
    df = df.drop(columns=["time"])
    #df.head()
    
    return df

In [10]:
df_train

,S.No,lat,lon,TMQ,U850,V850,UBOT,VBOT,QREFHT,PS,...,T200,T500,PRECT,TS,TREFHT,Z1000,Z200,ZBOT,time,LABELS
0,0,-24.758801,242.1875,16.019615,-4.391696,4.777769,-6.388222,7.725320,0.010277,102326.2344,...,221.594223,261.823212,6.660000e-09,295.622253,293.301514,197.698624,12234.78809,64.372635,20030804,0
1,1,23.820078,277.8125,47.802036,8.623652,9.308566,4.596105,9.938286,0.018531,100803.1094,...,220.343719,269.106720,3.670000e-16,302.496826,302.654083,71.571518,12463.97266,66.834389,20071006,1
2,2,23.820078,276.8750,11.556691,-2.483993,-6.009627,-3.503036,-5.921963,0.007468,101885.3906,...,215.933487,259.610718,0.000000e+00,299.854919,293.339996,160.414246,12099.36133,64.209663,20020331,0
3,3,13.494133,253.1250,53.186630,0.150933,-1.319407,3.757741,-2.172120,0.018809,100824.6328,...,218.457672,267.374329,9.800000e-08,301.606934,299.973450,72.840614,12379.26855,66.243294,20000906,0
4,4,-24.524120,241.2500,23.353998,-7.467506,-5.113565,-9.545109,-4.691221,0.011672,102553.8984,...,214.316605,263.575409,6.410000e-09,295.819488,295.101257,218.449570,12261.22656,64.876427,20000906,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
47755,47755,-30.625815,353.4375,14.680613,3.153518,4.250256,-0.406992,3.480553,0.008333,102109.4844,...,213.804184,259.846222,2.470000e-09,291.791870,290.141663,177.677322,12058.16016,63.678669,20001106,2
47756,47756,-24.993481,242.1875,29.540585,9.181329,-0.369805,7.125068,-5.466469,0.013645,101734.2813,...,213.867416,263.996307,2.620000e-09,296.322327,296.220642,150.108871,12272.25781,65.238327,20010712,2
47757,47757,-30.625815,353.4375,20.991110,7.709649,-6.284292,3.702500,-7.359541,0.010529,101800.8516,...,216.394852,264.192566,4.240000e-09,290.757996,290.736847,152.666443,12232.26953,63.929737,20011029,0
47758,47758,23.585398,276.5625,45.689816,-3.018448,0.280351,-6.938124,0.681055,0.020653,101641.2266,...,217.026199,265.778900,8.620000e-08,302.751862,301.542786,145.255554,12404.29297,66.656548,20020712,0


In [11]:
df_train = df_year_handler(df_train)

In [12]:
labels = df_train[["LABELS"]]

In [13]:
df_train = df_train.drop(columns=["S.No","LABELS"])

In [14]:
df_train = pd.concat([df_train, labels],axis=1)

In [15]:
df_train

,lat,lon,TMQ,U850,V850,UBOT,VBOT,QREFHT,PS,PSL,...,PRECT,TS,TREFHT,Z1000,Z200,ZBOT,year,month,date,LABELS
0,-24.758801,242.1875,16.019615,-4.391696,4.777769,-6.388222,7.725320,0.010277,102326.2344,102326.2344,...,6.660000e-09,295.622253,293.301514,197.698624,12234.78809,64.372635,2003,8,4,0
1,23.820078,277.8125,47.802036,8.623652,9.308566,4.596105,9.938286,0.018531,100803.1094,100803.1094,...,3.670000e-16,302.496826,302.654083,71.571518,12463.97266,66.834389,2007,10,6,1
2,23.820078,276.8750,11.556691,-2.483993,-6.009627,-3.503036,-5.921963,0.007468,101885.3906,101885.3906,...,0.000000e+00,299.854919,293.339996,160.414246,12099.36133,64.209663,2002,3,31,0
3,13.494133,253.1250,53.186630,0.150933,-1.319407,3.757741,-2.172120,0.018809,100824.6328,100824.6328,...,9.800000e-08,301.606934,299.973450,72.840614,12379.26855,66.243294,2000,9,6,0
4,-24.524120,241.2500,23.353998,-7.467506,-5.113565,-9.545109,-4.691221,0.011672,102553.8984,102553.8984,...,6.410000e-09,295.819488,295.101257,218.449570,12261.22656,64.876427,2000,9,6,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
47755,-30.625815,353.4375,14.680613,3.153518,4.250256,-0.406992,3.480553,0.008333,102109.4844,102109.4844,...,2.470000e-09,291.791870,290.141663,177.677322,12058.16016,63.678669,2000,11,6,2
47756,-24.993481,242.1875,29.540585,9.181329,-0.369805,7.125068,-5.466469,0.013645,101734.2813,101734.2813,...,2.620000e-09,296.322327,296.220642,150.108871,12272.25781,65.238327,2001,7,12,2
47757,-30.625815,353.4375,20.991110,7.709649,-6.284292,3.702500,-7.359541,0.010529,101800.8516,101800.8516,...,4.240000e-09,290.757996,290.736847,152.666443,12232.26953,63.929737,2001,10,29,0
47758,23.585398,276.5625,45.689816,-3.018448,0.280351,-6.938124,0.681055,0.020653,101641.2266,101641.2266,...,8.620000e-08,302.751862,301.542786,145.255554,12404.29297,66.656548,2002,7,12,0


In [17]:
latest_labels = df_train[["LABELS"]]

In [18]:
def df_to_train_test_split(df):
    #df is the normalized or standardized df
    #labels are just the labels.
#         So we need 6760 test exmaples, in 18:4:1 ratio roughly to match the train test.<br>
#         So we need:<br>
#         1s - 294<br>
#         2s - 1176<br>
#         0s - 5290<br>
    data_with_labels = df.sample(frac=1).reset_index(drop=True)
    
    data_with_labels_0s = data_with_labels[data_with_labels["LABELS"] == 0]
    data_with_labels_1s = data_with_labels[data_with_labels["LABELS"] == 1]
    data_with_labels_2s = data_with_labels[data_with_labels["LABELS"] == 2]

    data_1_test = data_with_labels_1s.iloc[0:294]
    data_2_test = data_with_labels_2s.iloc[0:1176]
    data_0_test = data_with_labels_0s.iloc[0:5290]

    data_1_train = data_with_labels_1s.iloc[294:]
    data_2_train = data_with_labels_2s.iloc[1176:]
    data_0_train = data_with_labels_0s.iloc[5290:]

    test = pd.concat([data_1_test, data_2_test, data_0_test])
    #Houston, we have a problem, we need to shuffle the indices, but pandas has us covered
    #https://stackoverflow.com/questions/29576430/shuffle-dataframe-rows
    train = pd.concat([data_1_train, data_2_train, data_0_train])

    #Shuffling:
    #df = df.sample(frac=1).reset_index(drop=True)
    test = test.sample(frac=1).reset_index(drop=True)
    train = train.sample(frac=1).reset_index(drop=True)

    #Finally, we have our train and test sets.
    #Let's try to follow this post: https://github.com/rahulrrai/multinomial-logistic-regression/blob/master/multinomial-logistic-regression.ipynb
    train_np = train.to_numpy()
    test_np = test.to_numpy()

    #number of training examples
    #this is 41000 in our case
    num_of_train = train_np.shape[0]
    num_of_test = test_np.shape[0]
    #retaining just the features
    X_train = train_np[:,:-1]
    X_test = test_np[:,:-1]
    ##As we are using vector implementation adding a column in X with all the value 1
    # I'm guessing this is the bias term added to our train matrix itself
#     X_train = np.hstack((np.ones((num_of_train,1)),X_train))
#     X_test = np.hstack((np.ones((num_of_test,1)), X_test))
    #output 
    y_train = train_np[:,-1:]
    y_test = test_np[:,-1:]
    #each individual entry in y is a numpy array, let's see if this works out.
    
    return X_train, X_test, y_train, y_test

In [19]:
X_train, X_test, y_train, y_test = df_to_train_test_split(df_train)

In [20]:
def standardizer_new(X_train):
    df = pd.DataFrame(X_train)
    #df = df.drop(columns=[0])
    df = (df-df.mean())/df.std()
    X_train = df.to_numpy()
    #num_of_train = X_train.shape[0]
    #X_train = np.hstack((np.ones((num_of_train,1)),X_train))
    return X_train

In [21]:
X_train_std = standardizer_new(X_train)
X_train_std

array([[ 0.71719023, -0.38529306,  1.76324278, ...,  2.1027921 ,
         0.08566519,  1.57947044],
       [-0.79978435, -0.9640404 , -0.8908907 , ..., -1.72500093,
         0.47311942, -0.41065309],
       [-0.94328195, -0.65182144,  0.45692047, ..., -0.0845182 ,
         0.08566519,  0.91609593],
       ...,
       [-1.20977748,  2.06676805, -1.16269733, ..., -0.35793199,
         0.47311942,  0.36328384],
       [ 0.70694041, -0.40813835,  1.81651321, ..., -0.35793199,
         0.86057366,  1.46890802],
       [ 0.71719023, -0.39290815,  1.11742826, ..., -0.0845182 ,
         0.08566519,  0.91609593]])

In [22]:
y_train = y_train.astype(int)
y_test = y_test.astype(int)

In [23]:
X_test_std = standardizer_new(X_test)

In [24]:
X_train_std.shape

(41000, 21)

In [25]:
X_test_std.shape

(6760, 21)

In [26]:
y_train.shape

(41000, 1)

In [27]:
np.unique(y_train, return_counts=True)

(array([0, 1, 2]), array([32245,  1708,  7047], dtype=int64))

In [28]:
y_test.shape

(6760, 1)

In [29]:
np.unique(y_test, return_counts=True)

(array([0, 1, 2]), array([5290,  294, 1176], dtype=int64))

In [30]:
X_train_full_std = np.concatenate((X_train_std, X_test_std), axis=0)

In [31]:
y_train_full = np.concatenate((y_train, y_test), axis=0)

In [41]:
from imblearn.combine import SMOTETomek
from imblearn.under_sampling import RandomUnderSampler

In [42]:
np.unique(y_train, return_counts=True)

(array([0, 1, 2]), array([32245,  1708,  7047], dtype=int64))

In [43]:
# Combining undersampling with oversampling since that gives better results

In [48]:
rus = RandomUnderSampler(sampling_strategy={0:22000,1:1708,2:7047},random_state=66)
X_temp, y_temp = rus.fit_resample(X_train, y_train)

In [49]:
np.unique(y_temp, return_counts=True)

(array([0, 1, 2]), array([22000,  1708,  7047], dtype=int64))

In [51]:
smt = SMOTETomek(sampling_strategy={0:22000, 1:5124, 2:14000}, random_state = 66)
X_train_smt, y_train_smt = smt.fit_resample(X_temp, y_temp)

In [52]:
np.unique(y_train_smt, return_counts=True)

(array([0, 1, 2]), array([21559,  5004, 13617], dtype=int64))

In [53]:
X_train_smt.shape

(40180, 21)

In [54]:
X_train_smt_std = standardizer_new(X_train_smt)
y_train_smt = y_train_smt

In [55]:
X_train_smt_std.shape

(40180, 21)

In [56]:
X_test_std.shape

(6760, 21)

In [57]:
y_train_smt.shape

(40180,)

In [58]:
y_test.ravel().shape

(6760,)

In [59]:
X_train_full_smt_std = np.concatenate((X_train_smt_std, X_test_std), axis=0)

In [60]:
y_train_full_smt = np.concatenate((y_train_smt, y_test.ravel()), axis=0)

In [275]:
# np.savetxt("X_train_full_smt_std.csv", X_train_full_smt_std, delimiter=",")
# np.savetxt("y_train_full_smt.csv", y_train_full_smt, delimiter=",")

In [67]:
param_grid1 = {
    'n_estimators' : [50, 100, 200, 300],
    'criterion': ['gini','entropy'],
    #removed None from below to avoid overfitting
    'max_depth' : [3, 5, 10],
    'min_samples_split' : [2, 4, 8, 20, 0.01, 0.1, 0.2, 0.4],
    'min_samples_leaf' : [1, 2, 4, 8, 16],
    'max_features' : [None, "sqrt", "log2"],
    'bootstrap' : [True, False],
    'class_weight' : [None, "balanced", "balanced_subsample"],
}

In [68]:
rf_clf = RandomForestClassifier()
grid1 = HalvingRandomSearchCV(estimator = rf_clf, param_distributions = param_grid1, refit = True, verbose = 2, n_jobs=-1, factor = 3)
# fitting the model for grid search 
grid1.fit(X_train_smt_std, y_train_smt.ravel())

n_iterations: 7
n_required_iterations: 7
n_possible_iterations: 7
min_resources_: 30
max_resources_: 40180
aggressive_elimination: False
factor: 3
----------
iter: 0
n_candidates: 1339
n_resources: 30
Fitting 5 folds for each of 1339 candidates, totalling 6695 fits
----------
iter: 1
n_candidates: 447
n_resources: 90
Fitting 5 folds for each of 447 candidates, totalling 2235 fits
----------
iter: 2
n_candidates: 149
n_resources: 270
Fitting 5 folds for each of 149 candidates, totalling 745 fits
----------
iter: 3
n_candidates: 50
n_resources: 810
Fitting 5 folds for each of 50 candidates, totalling 250 fits
----------
iter: 4
n_candidates: 17
n_resources: 2430
Fitting 5 folds for each of 17 candidates, totalling 85 fits
----------
iter: 5
n_candidates: 6
n_resources: 7290
Fitting 5 folds for each of 6 candidates, totalling 30 fits
----------
iter: 6
n_candidates: 2
n_resources: 21870
Fitting 5 folds for each of 2 candidates, totalling 10 fits


HalvingRandomSearchCV(estimator=RandomForestClassifier(), n_jobs=-1,
                      param_distributions={'bootstrap': [True, False],
                                           'class_weight': [None, 'balanced',
                                                            'balanced_subsample'],
                                           'criterion': ['gini', 'entropy'],
                                           'max_depth': [3, 5, 10],
                                           'max_features': [None, 'sqrt',
                                                            'log2'],
                                           'min_samples_leaf': [1, 2, 4, 8, 16],
                                           'min_samples_split': [2, 4, 8, 20,
                                                                 0.01, 0.1, 0.2,
                                                                 0.4],
                                           'n_estimators': [50, 100, 200, 300]},
                   

In [69]:
# print best parameter after tuning 
print(grid1.best_params_) 
grid_predictions = grid1.predict(X_test_std)

{'n_estimators': 200, 'min_samples_split': 4, 'min_samples_leaf': 1, 'max_features': 'sqrt', 'max_depth': 10, 'criterion': 'gini', 'class_weight': None, 'bootstrap': False}


In [70]:
print(classification_report(y_test, grid_predictions)) 

              precision    recall  f1-score   support

           0       0.97      0.85      0.91      5290
           1       0.65      0.88      0.75       294
           2       0.61      0.88      0.72      1176

    accuracy                           0.86      6760
   macro avg       0.74      0.87      0.79      6760
weighted avg       0.89      0.86      0.87      6760



In [71]:
print("Accuracy is:",accuracy_score(y_test, grid_predictions))
print("Balanced Accuracy is:",balanced_accuracy_score(y_test, grid_predictions))
print("F1 score is:",f1_score(y_test, grid_predictions, average = None))
print("F1 sample score is:",f1_score(y_test, grid_predictions, average = None))
print("Precision is:",precision_score(y_test, grid_predictions, average = None))
print("Recall is:",recall_score(y_test, grid_predictions, average = None))

#clf.predict_proba(X), multi_class='ovr'
#https://scikit-learn.org/stable/modules/generated/sklearn.metrics.roc_auc_score.html#sklearn.metrics.roc_auc_score
print("ROC AUC Score is:",roc_auc_score(y_test, grid1.predict_proba(X_test_std), multi_class = 'ovr'))

Accuracy is: 0.8594674556213018
Balanced Accuracy is: 0.8737531559567747
F1 score is: [0.90653266 0.75144509 0.72272411]
F1 sample score is: [0.90653266 0.75144509 0.72272411]
Precision is: [0.96781116 0.65326633 0.61104583]
Recall is: [0.85255198 0.88435374 0.88435374]
ROC AUC Score is: 0.9604769615620347


In [72]:
grid1.best_estimator_

RandomForestClassifier(bootstrap=False, max_depth=10, max_features='sqrt',
                       min_samples_split=4, n_estimators=200)

In [73]:
rf = RandomForestClassifier(bootstrap=False, max_depth=10, max_features='sqrt',
                       min_samples_split=4, n_estimators=200).fit(X_train_full_smt_std, y_train_full_smt.ravel())

In [74]:
#predictions = rf.predict()

In [75]:
submit_df = pd.read_csv(r"./ift3395-6390-weatherevents/test.csv")

In [76]:
submit_df = df_year_handler(submit_df)

In [77]:
submit_df = submit_df.drop(columns=["S.No"])

In [78]:
submit_df

,lat,lon,TMQ,U850,V850,UBOT,VBOT,QREFHT,PS,PSL,...,T500,PRECT,TS,TREFHT,Z1000,Z200,ZBOT,year,month,date
0,-31.095176,353.1250,22.372244,-1.994936,-0.002513,-7.732825,-1.864900,0.008203,102809.6719,102809.6719,...,258.135406,4.060000e-08,290.377930,289.014862,234.634003,12023.84082,63.400764,2013,9,27
1,-31.095176,353.4375,21.536032,-2.058918,0.397359,-8.436489,-0.940275,0.007887,102816.6016,102816.6016,...,258.205994,1.670000e-08,290.377228,288.873138,234.982056,12023.83301,63.348942,2013,9,27
2,-31.095176,353.7500,20.465317,-2.128485,0.717785,-8.842417,0.152449,0.007695,102827.8594,102827.8594,...,258.242279,1.220000e-08,290.377228,288.641480,235.631378,12025.41113,63.281322,2013,9,27
3,-31.095176,354.0625,19.967148,-2.126235,1.123356,-8.773702,1.251795,0.007519,102830.5391,102830.5391,...,258.335113,3.460000e-10,290.441406,288.420074,235.608460,12026.61426,63.216827,2013,9,27
4,-31.095176,354.3750,19.598663,-2.053118,1.823581,-8.472447,2.225254,0.007293,102823.8516,102823.8516,...,258.513702,5.920000e-10,290.468903,288.392975,235.004334,12027.29004,63.200485,2013,9,27
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7315,24.054759,276.5625,51.415295,-1.095974,-8.194263,2.484773,-10.520496,0.020928,100743.8438,100743.8438,...,268.880676,6.870000e-15,302.576569,302.226227,66.832077,12467.79492,66.832077,2013,9,16
7316,24.054759,276.8750,52.377407,-0.265653,-8.730537,3.783044,-10.748092,0.021389,100703.5313,100703.5313,...,269.085083,9.240000e-10,302.583557,302.131012,66.827492,12466.41309,66.827492,2013,9,16
7317,24.054759,277.1875,54.639217,0.775797,-9.646189,5.087689,-10.786784,0.021745,100673.9844,100673.9844,...,269.305939,5.870000e-08,302.646820,302.032715,66.812981,12467.56934,66.812981,2013,9,16
7318,24.054759,277.5000,56.121231,1.813888,-10.849813,6.442380,-10.859090,0.021840,100647.3359,100647.3359,...,269.616638,8.810000e-08,302.646820,302.014435,66.808281,12468.06934,66.808273,2013,9,16


In [79]:
def standardizer_new(X_train):
    df = pd.DataFrame(X_train)
    #df = df.drop(columns=[0])
    df = (df-df.mean())/df.std()
    X_train = df.to_numpy()
    #num_of_train = X_train.shape[0]
    #X_train = np.hstack((np.ones((num_of_train,1)),X_train))
    return X_train

In [80]:
X_submit = standardizer_new(submit_df)

In [81]:
X_submit.shape

(7320, 21)

In [82]:
predictions_submit = rf.predict(X_submit)

In [86]:
type(predictions_submit[0])

numpy.int32

In [83]:
predictions_df = pd.DataFrame(data=predictions_submit,columns=["LABELS"]).astype(int)
#https://stackoverflow.com/questions/18022845/pandas-index-column-title-or-name
predictions_df.index.name = "S.No"
predictions_df.to_csv(r"./ift3395-6390-weatherevents/submission_SMOTE_with_prior_undersampling_with_dups.csv")